In [ ]:
import numpy as np
import PIL
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
import glob2

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
data_dir = 'F:\TheNewNormal\data'
data_dir = pathlib.Path(data_dir) #Using pathlib to convert path to recognizable directory by system(os).

In [ ]:
batch_size = 35 #Creation of batches to reduce runtime load on CPU
image_height = 180
image_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2, #Using 20% of dataset to validate model.
  subset="training", #Pushing 80% images to training subset.
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, #Using 20% of dataset to validate model.
    subset="validation", #Pushing 20% of images to validation subset
    seed=123,
    image_size=(image_height, image_width),
    batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10,10)) #setting width and height of image to be displayed to 10X10.
for images, labels in train_ds.take(1):
    for i in range(6):
        ax = plt.subplot(3,2,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE) # Using prefetch(), during training, data preprocessing and model execution are combined.

In [ ]:
normalization_layer = layers.Rescaling(1./255) #Standarizing values to [0,1] for neural network.

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x),y)) #Using dataset.map() to overlap.
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0] #check pixel for first image.
print(np.min(first_image), np.max(first_image)) #Checking for pixel values.

In [ ]:
num_classes = 2

model = Sequential([
    layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),#Generates a tensor of outputs by convolving a convolution kernel with the layer input.
    layers.MaxPooling2D(), #Max pooling operation for 2D spatial data.
    layers.Conv2D(32, 3, padding='same', activation='relu'),#incresing number of filters.
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),#Again, increasing number of filters.
    layers.MaxPooling2D(),
    layers.Flatten(), #Flattens the ouput layer array.
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam', #Using the Adam optimization technique.
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #Calculates loss
             metrics=['accuracy'])

**Checking summary of model - Output Shapes and Paramaters.**

In [ ]:
model.summary()

**Training the model.**

In [ ]:
epochs = 9 #Used 10 epochs to get higher confidence level in testing.
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8,8)) 
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = "Validation Loss")
plt.legend(loc = 'upper right')
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
from PIL import Image
from IPython.display import display
img_path = './test/check2.jpg'

display(Image.open(img_path))

img = tf.keras.utils.load_img(
    img_path, target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely belongs to {} with a {:.2f} percent confidence"
     .format(class_names[np.argmax(score)], 100 * np.max(score))
     )